<a href="https://colab.research.google.com/github/trey3p/maritime-traffic/blob/main/counting_ships_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General code for counting ships, no specific region or time range specified

Importing

In [ ]:
import ee #Earth Engine
import datetime
import pandas as pd

Authenticating Earth Engine API

In [ ]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=vTfYdvYoHV_k982lXIjXG7hUhPWJgHu4aqCjtje8vrM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgAfC0Ajo9r56cFR5dOY6d8SCWQ-VrA5XN_R9hiP9MbND07V47Nc1g

Successfully saved authorization token.


Defining Region of Interest

In [ ]:
region = ee.Geometry.Polygon(
    #coordinates of region
)

Loading Sentinel-2 and Sentinel-1 Collections

In [ ]:
s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(region).select('B11').min().clip(region)

#Using Sentinel-2 satellite to build the water mask. The B11 band gives short-wave infrared imagery, 
#which is useful because water appears dark in SWIR. 
#The minimum is computed because we want to filter out objects in the water that have a higher pixel value than the water.

s1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(
    #time range
).select('VH')

#We use Sentinel-1 for actually detecting the ships. VH is dual-band cross-polarization, vertical transmit and horizontal receive.


NameError: ignored

Kernel

In [ ]:
kernel = ee.Kernel.circle({radius: 4});

Counting Ships Function

In [ ]:
def count_ships(sar_img):
  opened = sar_img.updateMask(water_mask).gt(-10).focal_max(kernel = kernel, iterations = 1).focal_min(kernel = kernel, iterations = 1)

  objectId = opened.select('VH').connectedComponents(connectedness = ee.Kernel.plus(1), maxSize = 128)

  objectSize = objectId.select('labels').connectedPixelCount(maxSize = 128, eightConnected = false)

  pixelArea = ee.Image.pixelArea()
  objectArea = objectSize.multiply(pixelArea)

  areaMask = objectArea.gte(1000)
  objectId = objectId.updateMask(areaMask)

  result = objectId.select('labels').reduceRegion(geometry = region, reducer = ee.Reducer.countDistinct(),
                                                  scale = 10, bestEffort = True)
  
  return ee.Feature(null, 'date' = ee.Date(sar_img.get('system:time_start'))
        .format("YYYY-MM-dd"),
      'count' = result.get('labels'))
  


Applying the Function

In [ ]:
collection = s1.map(count_ships)

Exporting Data

In [ ]:
def fc_df(feature_collection):
  features = feature_collection.getInfo()['features']

  dictarr = []

  for f in features:
    attr = f['properties']

    dictarr.append(attr)


  df = pd.DataFrame(dictarr)
  return df


In [ ]:
fc_df(collection).to_csv('ship-count-data.csv')